In [29]:
import pandas as pd
import numpy as np

In [30]:
#Criar dataframe
treino = pd.read_csv("train.csv")

In [31]:
#Removendo as colunas que não influenciam o modelo
treino = treino.drop(['Artist Name', 'Track Name', 'Popularity', 'duration_in min/ms'], axis=1)

In [32]:
#Verificando valores nulos nas colunas restantes
print(treino.isnull().sum())

danceability           0
energy                 0
key                 2014
loudness               0
mode                   0
speechiness            0
acousticness           0
instrumentalness    4377
liveness               0
valence                0
tempo                  0
time_signature         0
Class                  0
dtype: int64


In [33]:
#Imputando dados em 'key' com a moda de cada gênero
for i in range(11):
  moda = treino.loc[treino['Class'] == i, 'key'].mode()[0]
  print("Gênero", i, "tem moda", moda)
  treino.loc[(treino['Class'] == i) & (treino['key'].isna()), 'key'] = moda

Gênero 0 tem moda 2.0
Gênero 1 tem moda 7.0
Gênero 2 tem moda 7.0
Gênero 3 tem moda 7.0
Gênero 4 tem moda 2.0
Gênero 5 tem moda 1.0
Gênero 6 tem moda 9.0
Gênero 7 tem moda 2.0
Gênero 8 tem moda 1.0
Gênero 9 tem moda 1.0
Gênero 10 tem moda 2.0


In [34]:
#Limitando a coluna 'instrumentalness' para 4 casas decimais
treino['instrumentalness'] = treino['instrumentalness'].round(4)

#Imputando dados em 'instrumentalness' com a média de cada gênero
for i in range(11):
  media = treino.loc[treino['Class'] == i, 'instrumentalness'].mean()
  media = media.round(4)
  print("Gênero", i, "tem média", media)
  treino.loc[(treino['Class'] == i) & (treino['instrumentalness'].isna()), 'instrumentalness'] = media

Gênero 0 tem média 0.0995
Gênero 1 tem média 0.1992
Gênero 2 tem média 0.0979
Gênero 3 tem média 0.1314
Gênero 4 tem média 0.0127
Gênero 5 tem média 0.0688
Gênero 6 tem média 0.2626
Gênero 7 tem média 0.7404
Gênero 8 tem média 0.2359
Gênero 9 tem média 0.0498
Gênero 10 tem média 0.1128


In [35]:
#Verificando se preenchimento aconteceu com sucesso
print(treino.isnull().sum())

danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
Class               0
dtype: int64


In [36]:
#Verificando colunas existentes
treino.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature', 'Class'],
      dtype='object')

In [37]:
#Desenvolvendo modelo de ML
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

#Separando as características e o alvo
X = treino.drop('Class', axis=1)
y = treino['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

#Criando o modelo de árvore de decisão
clf = DecisionTreeClassifier(criterion = 'gini', splitter = 'best', max_depth = 10, min_samples_leaf = 1, min_samples_split = 10, random_state = 42)

#Treinando o modelo
clf.fit(X_train, y_train)

#Fazendo previsões com o conjunto de teste
y_pred = clf.predict(X_test)

#Calcular a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)

print(f'Acurácia do modelo: {accuracy}')

Acurácia do modelo: 0.5228939764392088


In [38]:
#Instalando a biblioteca do spotify para realizar classificação de músicas fornecidas pelo usuário

!pip install spotipy # - Executar quando iniciar pela primeira vez o colab

In [45]:
#Pegando musica diferente do spotify
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Credenciais da API
client_credentials_manager = SpotifyClientCredentials(client_id='9b1e9ff837434408ae773f93a6facb78', client_secret='67dbfcb123f343b2a6921dbfe243914f')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

#Obtendo o nome da música do usuário
nome_da_musica = input("Digite o nome da música que você quer pesquisar: ")
results = sp.search(q=nome_da_musica, limit=1)

#Obtendo o nome da música e do artista
musica = results['tracks']['items'][0]['name']
autor = results['tracks']['items'][0]['artists'][0]['name']

#Mostrando o resultado
print("A música que você buscou retornou o resultado: ", musica, " - ", autor)

#Pegando o ID da música encontrada
id_da_musica = results['tracks']['items'][0]['id']

#Obtendo os dados da música
dadosDaMusica = sp.audio_features(id_da_musica)[0]

#Limitando o atributo 'instrumentalness' para apenas 4 casas decimais
dadosDaMusica['instrumentalness'] = round(dadosDaMusica['instrumentalness'], 4)


Digite o nome da música que você quer pesquisar: Better Things - aespa
A música que você buscou retornou o resultado:  Better Things  -  aespa


In [46]:
# Mapeando os dados fornecidos para a ordem desejada
caracteristicas_relevantes = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']
valores = [dadosDaMusica[key] for key in caracteristicas_relevantes]

# Usando a lista de valores para a previsão
predicao = clf.predict([valores])

if predicao == 0:
    categoria_correspondente = "Acoustic/Folk"
elif predicao == 1:
    categoria_correspondente = "Alt Music"
elif predicao == 2:
    categoria_correspondente = "Blues"
elif predicao == 3:
    categoria_correspondente = "Bollywood"
elif predicao == 4:
    categoria_correspondente = "Country"
elif predicao == 5:
    categoria_correspondente = "HipHop"
elif predicao == 6:
    categoria_correspondente = "Indie/Alt"
elif predicao == 7:
    categoria_correspondente = "Instrumental"
elif predicao == 8:
    categoria_correspondente = "Metal"
elif predicao == 9:
    categoria_correspondente = "Pop"
elif predicao == 10:
    categoria_correspondente = "Rock"
else:
    categoria_correspondente = "Categoria Desconhecida"

#Printando resultado
print("A musica", musica, "de", autor, "faz parte do Gênero Musical -", categoria_correspondente, predicao)

A musica Better Things de aespa faz parte do Gênero Musical - Pop [9]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [48]:
#Obtendo recomendações
recommendations = sp.recommendations(seed_tracks=[id_da_musica],
                                      limit=10,
                                      target_danceability=dadosDaMusica['danceability'],
                                      target_energy=dadosDaMusica['energy'],
                                      target_liveness=dadosDaMusica['liveness'],
                                      target_valence=dadosDaMusica['valence'])


# Imprimir os nomes das músicas recomendadas e seus artistas
print("Músicas com a mesma \"vibe\": \n ")
for track in recommendations['tracks'][1:]:
    print(f"\t Música: {track['name']} — {track['artists'][0]['name']} \n")

Músicas com a mesma "vibe": 
 
	 Música: God's Menu — Stray Kids 

	 Música: Good Parts (when the quality is bad but I am) — LE SSERAFIM 

	 Música: TWIT — HWASA 

	 Música: Rumor — H.I.N.P (Hot Issue of Ntl. Producers) 

	 Música: RHYTHM TA — iKON 

	 Música: 24 Hours — SUNMI 

	 Música: BOY — TREASURE 

	 Música: Illusion — aespa 

	 Música: LOVE DIVE — IVE 

